<a href="https://colab.research.google.com/github/rze7ud/DS3001_Final_Project/blob/main/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install python-dotenv
!pip install cfbd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 4.2 MB/s eta 0:00:00


In [36]:
import cfbd
from cfbd.rest import ApiException
from __future__ import print_function
import json

import pandas as pd

from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv('API_KEY')

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = API_KEY
configuration.api_key_prefix['Authorization'] = 'Bearer'

In [35]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
year = 2024
team = "Clemson"

try:
    api_response = api_instance.get_games(year=year, team=team)
except ApiException as e:
    print("Exception when calling BettingApi->get_lines: %s\n" % e)

In [40]:
columns = ['year', 'week', 'home', 'clemson_points', 'opponent', 'opponent_points']
df = pd.DataFrame(columns=columns)

for game in api_response:
    year = game.season
    week = game.week
    if game.home_team == "Clemson":
        home = 'True'
        clemson_points = game.home_points
        opponent = game.away_team
        opponent_points = game.away_points
    else:
        home = 'False'
        clemson_points = game.away_points
        opponent = game.home_team
        opponent_points = game.home_points
    df.loc[len(df)] = [year, week, home, clemson_points, opponent, opponent_points]